# Lambda calculus in Kotlin

## Basic interface definition

Since `kotlin.FunctionN` is not able to express a recursive lambda type, whose parameter and return type are both itself, we define a new interface `Lambda` to express it.

In [1]:
fun interface Lambda {
    operator fun invoke(f: Lambda): Lambda
}

## Shorthand for defining a lambda expression with currying

If we want to write `λab.a` in Kotlin, embedded braces like `{ a -> { b -> a } }` are annoying.
Thus, we can use the power of currying.

In [2]:
fun lambda(block: (Lambda) -> Lambda) =
    Lambda { p0 -> block(p0) }

fun lambda(block: (Lambda, Lambda) -> Lambda) =
    Lambda { p0 -> lambda { p1 -> block(p0, p1) } }

fun lambda(block: (Lambda, Lambda, Lambda) -> Lambda) =
    Lambda { p0 -> lambda { p1, p2 -> block(p0, p1, p2) } }

fun lambda(block: (Lambda, Lambda, Lambda, Lambda) -> Lambda) =
    Lambda { p0 -> lambda { p1, p2, p3 -> block(p0, p1, p2, p3) } }

fun lambda(block: (Lambda, Lambda, Lambda, Lambda, Lambda) -> Lambda) =
    Lambda { p0 -> lambda { p1, p2, p3, p4 -> block(p0, p1, p2, p3, p4) } }

fun lambda(block: (Lambda, Lambda, Lambda, Lambda, Lambda, Lambda) -> Lambda) =
    Lambda { p0 -> lambda { p1, p2, p3, p4, p5 -> block(p0, p1, p2, p3, p4, p5) } }

## Natural numbers

Then we can define natural numbers naturally.

In [3]:
val zero = lambda { f, x -> x }
val one = lambda { f, x -> f(x) }
val two = lambda { f, x -> f(f(x)) }
val three = lambda { f, x -> f(f(f(x))) }
val four = lambda { f, x -> f(f(f(f(x)))) }
val five = lambda { f, x -> f(f(f(f(f(x))))) }

To make it visible, we can define a function to transform from Church numbers to `kotlin.Int`.

In [4]:
val identity = lambda { it -> it }

fun Lambda.toInt(): Int {
    var count = 0
    this { count++; it }(identity)
    return count
}

And the reversed transform:

In [5]:
fun Int.toChurch(): Lambda =
    if (this <= 0) zero
    else Lambda { f -> Lambda { x -> f((this - 1).toChurch()(f)(x)) } }

Let's see the output:

In [6]:
println(zero.toInt())
println(one.toInt())
println(two.toInt())
println(0.toChurch().toInt())
println(100.toChurch().toInt())

0
1
2
0
100


It matches our expectation.

## Basic arithmetics

Let's start from the simplest ones.

In [7]:
val successor = lambda { n, f, x -> f(n(f)(x)) }
val multiply = lambda { m, n, f -> m(n(f)) }
val pow = lambda { a, b -> b(a) }

And let's define operators in Kotlin for them.

In [8]:
operator fun Lambda.plus(other: Lambda): Lambda = this(successor)(other)
operator fun Lambda.times(other: Lambda): Lambda = multiply(this)(other)

Let's test them.

In [9]:
println((one + two * three).toInt())

7


When it comes to substraction, things are bit of more complex.

We need a pair `(0, 0)`. Then we define a `Φ` combinator, which moves a pair `(a, b)` to `(b, b+1)`.
We invoke `Φ` for `n` times on `(0, 0)`, and we get `(n-1, n)` in the end.
Finally, we only need to get the first element of the pair. Then we get `n-1` from `n`.

In [10]:
val pair = lambda { a, b, f -> f(a)(b) }
val first = lambda { a, b -> a }
val second = lambda { a, b -> b }
val fai = lambda { p -> pair(p(second))(successor(p(second))) }

Let's test out pair.

In [11]:
{
    val myPair = pair(10.toChurch())(20.toChurch())
    println(myPair(first).toInt())
    println(myPair(second).toInt())
    println(fai(myPair)(first).toInt())
    println(fai(myPair)(second).toInt())
}()

10
20
20
21


It seems that `pair` and `fai` works correctly.

Now we can write the substraction.

In [12]:
val predecessor = lambda { n -> n(fai)(pair(zero)(zero))(first) }
val subtract = lambda { a, b -> b(predecessor)(a) }
operator fun Lambda.minus(other: Lambda): Lambda = subtract(this)(other)

Let's test it.

In [13]:
println((25.toChurch() - 13.toChurch()).toInt())

12


## Boolean arithmetics

We need to define `true` and `false` first.

In [14]:
val `true` = lambda { a, b -> a }
val `false` = lambda { a, b -> b }

Have you noticed? `true` is same as `first` of `pair` and `false` is same as `second` of `pair`.

Then, we can define basic logic operators.

In [15]:
val and = lambda { a, b -> a(b)(a) }
val or = lambda { a, b -> a(a)(b) }
val not = lambda { a -> a(`false`)(`true`) }
val xor = lambda { a, b -> a(not(b))(b) }

Confused? Me too.

They are kind of tricky things, so we need to check if they are right. Make it visible!

In [18]:
fun Lambda.toBoolean(): Boolean {
    var result = false
    this(lambda { it -> result = true;it })(lambda { it -> result = false;it })(identity)
    return result
}

In [19]:
println("--- NOT ---")
println("not(true) = ${not(`true`).toBoolean()}")
println("not(false) = ${not(`false`).toBoolean()}")
println()

println("--- AND ---")
println("true and true = ${and(`true`)(`true`).toBoolean()}")
println("true and false = ${and(`true`)(`false`).toBoolean()}")
println("false and true = ${and(`false`)(`true`).toBoolean()}")
println("false and false = ${and(`false`)(`false`).toBoolean()}")
println()

println("--- OR ---")
println("true or true = ${or(`true`)(`true`).toBoolean()}")
println("true or false = ${or(`true`)(`false`).toBoolean()}")
println("false or true = ${or(`false`)(`true`).toBoolean()}")
println("false or false = ${or(`false`)(`false`).toBoolean()}")
println()

println("--- XOR ---")
println("true xor true = ${xor(`true`)(`true`).toBoolean()}")
println("true xor false = ${xor(`true`)(`false`).toBoolean()}")
println("false xor true = ${xor(`false`)(`true`).toBoolean()}")
println("false xor false = ${xor(`false`)(`false`).toBoolean()}")

--- NOT ---
not(true) = false
not(false) = true

--- AND ---
true and true = true
true and false = false
false and true = false
false and false = false

--- OR ---
true or true = true
true or false = true
false or true = true
false or false = false

--- XOR ---
true xor true = false
true xor false = true
false xor true = true
false xor false = false


After boolean calculations, do you want `if` and `else`? Let's do it!

In [20]:
val `if` = lambda { c, t, f -> c(t)(f) }

In [22]:
println("if (true) 25 else 10 = ${`if`(`true`)(25.toChurch())(10.toChurch()).toInt()}")
println("if (false) 25 else 10 = ${`if`(`false`)(25.toChurch())(10.toChurch()).toInt()}")

if (true) 25 else 10 = 25
if (false) 25 else 10 = 10


## Recursive

After mastering these basic operations, it's time to try some high-level things!

In [28]:
{

}()

java.lang.StackOverflowError: 